# Plate, Account, and Tag Frequencies

This notebook explores the number of trips per plate, account and tags, and tries to explain these numbers.

In [1]:
suppressWarnings(suppressMessages(library(tidyverse)))
suppressMessages(library(lubridate))
library(RSQLCipher)

In [2]:
Sys.setenv("SQL_KEY"=Sys.getenv("HOT_KEY"))

In [3]:
db_path = "../../../data/hot.db"

# import tables
bos = load_table(db_path, "bos", c(tag_id="c", posted_account="c", 
                                   plate_state_pri="c", plate_state_sec="c")) # ~10M rows

In [4]:
replace_null = function(x) {
    na_if(na_if(na_if(x, "11183834060272721597"), "-8355759756528748941"), "8974271441158017554")
}

## Plates

Since there are 365 days in the year, we'd expect most accounts/plates to have fewer than 700 trips a year.  Anything above that is cause for suspicion.  Looking at the most frequent plates, we see that some plates are registered over 50,000, 11,000, and 8,000 times. There are several plates in the 1500-4000 range, too.

In [54]:
bos %>%
    group_by(id=plate_state_sec) %>%
    summarize(count=n()) %>%
    arrange(desc(count)) %>%
    head(15) %>%
    execute(col_types="ci") %>%
    mutate(id=replace_null(id))

id,count
<chr>,<int>
NA,732140
NA,51168
-8785122124138371598,11735
-3384025460270323568,8976
-6167837656755351917,3863
-1429594590878077513,3269
-5959073759183277597,2598
-4302454808096330693,2193
-7626160874510872818,1543


In [58]:
bos %>%
    group_by(id=plate_state_pri) %>%
    summarize(count=n()) %>%
    arrange(desc(count)) %>%
    head(15) %>%
    execute(col_types="ci") %>%
    mutate(id=replace_null(id))

id,count
<chr>,<int>
NA,51168
-3384025460270323568,11073
-1429594590878077513,2462
-987962521958205811,1983
-5959073759183277597,1911
-7875715471405424454,1778
1978784222145781431,1519
-4302454808096330693,1363
-5828082255304641318,1057


The BOS file is joined to the census/account file using `posted_account`, `plate_state_sec`, and `plate_state_pri`.  As long as the combination of these is unique, we need not be concerned about duplicate plates and accounts.  

Doing this, the most-frequent plate was now used "only" around 10,000 times in 2018 (8365 times when the plate was processed completely, and nother 2188 times when the plate was partially processed). 

Beyond this problematic plate, usage patterns make sense.

In [59]:
bos %>%
    group_by(acct=posted_account, plate=plate_state_sec, plate_pri=plate_state_pri) %>%
    summarize(count=n()) %>%
    arrange(desc(count)) %>%
    head(15) %>%
    execute(col_types="ccci") %>%
    mutate(plate=replace_null(plate), plate_pri=replace_null(plate_pri))

acct,plate,plate_pri,count
<chr>,<chr>,<chr>,<int>
NA,-3384025460270323568,-3384025460270323568,8365
-3507430751919157279,NA,NA,4824
-7228328777495153646,NA,NA,2427
NA,NA,-3384025460270323568,2188
NA,NA,NA,1382
-7893186328889783179,-372349001783834727,-372349001783834727,848
953887187898708034,-6644171243177243892,-6644171243177243892,755
656702052906128553,-6493262359384201288,-6493262359384201288,696
3262768471685963954,5630667284520023488,5630667284520023488,652


We can also try to tease apart these uses by `tag_id`, which should in theory be unique to a vehicle. This problem is the same single problematic plate, "-3384025460270323568". 

In [60]:
bos %>%
    group_by(acct=posted_account, plate=plate_state_sec, plate_pri=plate_state_pri, tag_id) %>%
    summarize(count=n()) %>%
    arrange(desc(count)) %>%
    head(15) %>%
    execute(col_types="cccci") %>%
    mutate(plate=replace_null(plate), plate_pri=replace_null(plate_pri), tag_id=replace_null(tag_id))

acct,plate,plate_pri,tag_id,count
<chr>,<chr>,<chr>,<chr>,<int>
NA,-3384025460270323568,-3384025460270323568,NA,8365
NA,NA,-3384025460270323568,NA,2188
-7893186328889783179,-372349001783834727,-372349001783834727,NA,848
953887187898708034,-6644171243177243892,-6644171243177243892,-8496308514258509090,733
1680008575170345795,373812370003301869,373812370003301869,-8010875764786662644,637
-730087134288330736,-8362674334327864509,-8362674334327864509,3614041388218899973,628
-3129791676948063044,-5562192719766814090,-5562192719766814090,NA,618
8753495716351625936,-6370314988642211106,-6370314988642211106,NA,614
8753495716351625936,2003741751969406404,2003741751969406404,NA,572


We can grab some sample rows from the `bos` table with these problematic plates

In [61]:
bos %>%
    mutate(plate_state_pri = as.character(plate_state_pri)) %>%
    filter(plate_state_pri == "-3384025460270323568") %>%
    head(2) %>%
    execute(col_types="iiiccccccii")

txn_id,trip_id,veh_class,tag_id,posted_account,plate_state_pri,plate_state_sec,plate_state,lic_plate_type_code,zip_code,plus4_code
<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
246543669,104027829,2,NA,NA,-3384025460270323568,-3384025460270323568,WA,NA,NA,NA
246543674,104027830,2,NA,NA,-3384025460270323568,-3384025460270323568,WA,NA,NA,NA


## Accounts

We see many accounts with a large number of trips, but this is less of a concern, since we would expect multiple vehicles to be associated to single accounts such as transit agencies, large commercial customers, etc.

In [56]:
bos %>%
    group_by(id=posted_account) %>%
    summarize(count=n()) %>%
    arrange(desc(count)) %>%
    head(15) %>%
    execute(col_types="ci") %>%
    mutate(id=replace_null(id))

id,count
<chr>,<int>
NA,1554847
-969745003671130354,106600
8753495716351625936,85112
-1778903698080337918,68843
-1385090625309080303,66475
-3507430751919157279,31826
-7228328777495153646,31474
-3271969786727291019,19982
-6448167887457587666,14791


Some of these accounts have over 25,000 plates associated with them.  Many have over 1,000 plates.

In [69]:
bos %>%
    select(acct=posted_account, plate_state_sec) %>%
    distinct() %>%
    group_by(acct) %>%
    summarize(plates=n()) %>%
    arrange(desc(plates)) %>%
    head(15) %>%
    execute(col_types="ci") 

acct,plates
<chr>,<int>
NA,461129
-1778903698080337918,25881
8753495716351625936,17767
-7228328777495153646,12085
-3507430751919157279,9377
-969745003671130354,8919
-6448167887457587666,7792
-1385090625309080303,2746
-2870755613724292448,2515


We get a better picture by looking at tags.  The largest account has 2,300 tags.  King County Metro's fleet has 1,500 buses, so these are on the righ order of magnitude.

In [77]:
bos %>%
    select(acct=posted_account, tag_id) %>%
    distinct() %>%
    group_by(acct) %>%
    summarize(tags=n()) %>%
    arrange(desc(tags)) %>%
    head(15) %>%
    execute(col_types="ci") 

acct,tags
<chr>,<int>
NA,16453
-2870755613724292448,2302
-969745003671130354,912
-3507430751919157279,833
8753495716351625936,553
4718396806487642257,405
-3271969786727291019,360
-7228328777495153646,335
-1385090625309080303,311


Looking at the number of tags per account, we see that there are 200,000 accounts with just one tag, and 2,277 accounts with five or more tags.

In [80]:
bos %>%
    select(acct=posted_account, tag_id) %>%
    distinct() %>%
    group_by(acct) %>%
    summarize(tags=n()) %>%
    arrange(desc(tags)) %>%
    summarize(n1=sum(tags==1), n2=sum(tags==2), n3=sum(tags==3), 
              n4=sum(tags==4), n5=sum(tags==5), ng5l10=sum(tags>5 & tags<=10),
              ng10l50=sum(tags>10 & tags<=50), ng50l100=sum(tags>50 & tags <=100),
              ng100=sum(tags>100)) %>%
    execute

n1,n2,n3,n4,n5,ng5l10,ng10l50,ng50l100,ng100
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
205426,68153,20003,5377,1602,1491,742,27,17


In [10]:
trips = load_table(db_path, "trips_linked", c(tag_id="c", acct="c", plate="c", id="c"))

In [16]:
comm_acct = bos %>%
    select(acct=posted_account, tag_id) %>%
    distinct() %>%
    group_by(acct) %>%
    summarize(commercial=n() > 6) 

left_join(select(trips, acct),
         comm_acct,
         by="acct") %>%
group_by(commercial) %>%
summarize(trips=n()) %>%
execute

Only 57% of commericial accounts have census linkages, compared to 76% of noncommercial accounts.

In [15]:
left_join(select(trips, acct, fips),
         comm_acct,
         by="acct") %>%
group_by(commercial, is.na(fips)) %>%
summarize(pct=n()) %>%
execute

commercial,is.na(fips),pct
<dbl>,<dbl>,<dbl>
NA,0,652864
NA,1,7807574
0,0,5527071
0,1,1727472
1,0,717994
1,1,543159


## Summary

Frequency-of-use patterns look relatively good, overall.  There are many accounts which used the system thousands or even tens of thousands of times in 2018.  These are likely commercial or government users.  Excluding `NULL` plates, there is just one plate/account pair which was logged in the system more than 1,000 times.  Plates logged less than 1,000 times correspond to 2-3 trips on the facility per day, which does not seem too unusual.  

The single plate which was logged 10,553 times (an average of around 28 times a day) does not have any account or ZIP code information.  It is a Washington state plate with no particular special type.  As this plate occured in the `bos` table, we know it paid a fare for each trip.  Sample transaction IDs to look up for future investigation are `246543669` and `246543674`.

Several accounts have tens of thousands of associated plates.  This is potentially concerning, as it could indicate a poor OCR plate recognition rate.  The largest account has around 2,500 Good to Go tags associated with it, which is plausible. Overall, 0.7% of accounts have more than six tags.  These accounts made up around 15% of account trips, and 7% of overall trips.  These could perhaps be classified as commercial users.